## Hybrid Quantum-Classical Neural Networks (HQCNNs)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from qiskit.primitives import StatevectorEstimator
from qiskit.circuit.library import RealAmplitudes
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector


In [3]:
# Step 1: Generate and Preprocess Data
X, y = make_moons(n_samples=100, noise=0.1, random_state=42)

# Normalize and split data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
X_test_torch = torch.tensor(X_test, dtype=torch.float)
y_train_torch = torch.tensor(y_train,dtype=torch.long)
y_test_torch = torch.tensor(y_test, dtype=torch.long)


In [4]:
# Step 2: Define the Quantum Circuit (RealAmplitudes)
num_qubits = 2
quantum_circuit = RealAmplitudes(num_qubits, reps=3)

In [5]:
# Step 3: Wrap the Quantum Circuit in an EstimatorQNN
estimator = StatevectorEstimator()
qnn = EstimatorQNN(
    circuit=quantum_circuit,
    estimator=estimator,
    input_params=quantum_circuit.parameters[:2],
    weight_params=quantum_circuit.parameters[2:]
)

No gradient function provided, creating a gradient function. If your Estimator requires transpilation, please provide a pass manager.


In [6]:
# Step 4: Convert Quantum Circuit to a Pytorch-Compatible Layer
quantum_layer = TorchConnector(qnn)

In [7]:
# Step 5: Define Hybrid Quantum-Classical Neural Network
class HybridQuantumNN(nn.Module):
    def __init__(self):
        super(HybridQuantumNN, self).__init__()
        self.fc1 = nn.Linear(2,2)
        self.quantum = quantum_layer
        self.fc2 = nn.Linear(1,2)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = self.quantum(x)
        x = self.fc2(x)
        return x

In [ ]:
# Step 6: Initialize Model, Loss Function, and Optimizer
model = HybridQuantumNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [12]:
# Step 7: Train the Model
epochs = 50
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train_torch)
    loss = criterion(outputs, y_train_torch)
    loss.backward()
    optimizer.step()

    if (epoch +1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

Epoch 10/50, Loss: 0.6826
Epoch 20/50, Loss: 0.6684
Epoch 30/50, Loss: 0.6408
Epoch 40/50, Loss: 0.6019
Epoch 50/50, Loss: 0.5624


In [13]:
# Step 8: Evaluate the Model
with torch.no_grad():
    y_pred = model(X_test_torch).argmax(dim=1)
    accuracy = (y_pred == y_test_torch).float().mean()
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 85.00%
